# Pretraining tail network

In [1]:
from src.dlc_practical_prologue import generate_pair_sets
from src.utils import load_class_data, load_target_data, load_all_data, print_param_count
from src.models import *
from src.trainer import Trainer
import torch
import matplotlib.pyplot as plt

In [2]:
# Load data with class and larget than as targets
dl_train_all, dl_val_all, dl_test_all = load_all_data()

# Load data with class as target
dl_train_class, dl_val_class, dl_test_class = load_class_data()

In [3]:
print("--- Pre-training tail model ---\n")

tail = TailLinear(label_encoded=True)
trainer = Trainer(nb_epochs=5, run='pretrained_tail_tailTraining')
trainer.fit(tail, dl_train_all, dl_val_all)

print("\n--- Training full model ---\n")

tail.requires_grad_=False
le_net = LeNet()
siamese = Siamese(le_net, tail, softmax=False, strategy='sum')
trainer = Trainer(nb_epochs=20, run='pretrained_tail_headTraining')
trainer.fit(siamese, dl_train_all, dl_val_all)

print("\n--- Testing model ---\n")
trainer.test(le_net, dl_test_class, return_acc=False)
trainer.test(siamese, dl_test_all, return_acc=False)


--- Pre-training tail model ---

# Epoch 1/5:	 loss=0.67	 loss_val=0.63	 acc_val=85.71
# Epoch 2/5:	 loss=0.55	 loss_val=0.43	 acc_val=91.52
# Epoch 3/5:	 loss=0.29	 loss_val=0.18	 acc_val=100.0
# Epoch 4/5:	 loss=0.11	 loss_val=0.07	 acc_val=100.0
# Epoch 5/5:	 loss=0.05	 loss_val=0.03	 acc_val=100.0

--- Training full model ---

# Epoch 1/20:	 loss=1.96	 loss_val=0.87	 acc_val=74.11
# Epoch 2/20:	 loss=0.89	 loss_val=0.24	 acc_val=93.75
# Epoch 3/20:	 loss=0.5	 loss_val=0.23	 acc_val=94.2
# Epoch 4/20:	 loss=0.44	 loss_val=0.2	 acc_val=95.54
# Epoch 5/20:	 loss=0.32	 loss_val=0.17	 acc_val=96.88
# Epoch 6/20:	 loss=0.27	 loss_val=0.19	 acc_val=95.98
# Epoch 7/20:	 loss=0.22	 loss_val=0.16	 acc_val=96.88
# Epoch 8/20:	 loss=0.22	 loss_val=0.13	 acc_val=97.77
# Epoch 9/20:	 loss=0.15	 loss_val=0.13	 acc_val=97.32
# Epoch 10/20:	 loss=0.13	 loss_val=0.14	 acc_val=96.88
# Epoch 11/20:	 loss=0.14	 loss_val=0.14	 acc_val=97.32
# Epoch 12/20:	 loss=0.09	 loss_val=0.1	 acc_val=97.77
# Epoch 

In [4]:
print_param_count(siamese)

Total number of parameters:     83300
Number of trainable parameters: 83300


In [5]:
# Do the same, but 10 times and with verbose=False
results = []
for i in range(10):
    print(f"Run {i+1}/10")

    # Load data with class and larget than as targets
    dl_train_all, dl_val_all, dl_test_all = load_all_data()

    # Load data with class as target
    dl_train_class, dl_val_class, dl_test_class = load_class_data()

    # Training tail model
    tail = TailLinear(label_encoded=True)
    trainer = Trainer(nb_epochs=5, run='pretrained_tail_tailTraining')
    trainer.fit(tail, dl_train_all, dl_val_all, verbose=False)

    # Training full model (but without gradient for last part)
    tail.requires_grad_=False
    le_net = LeNet()
    siamese = Siamese(le_net, tail, softmax=False, strategy='sum')
    trainer = Trainer(nb_epochs=20, run='pretrained_tail_headTraining_argmax')
    trainer.fit(siamese, dl_train_all, dl_val_all, verbose=False)

    # Testing
    acc = trainer.test(siamese, dl_test_all, test_verbose=True, return_acc=True)
    results.extend([acc])

Run 1/10
loss_test=0.16	 acc_test=97.36
Run 2/10
loss_test=0.16	 acc_test=96.97
Run 3/10
loss_test=0.19	 acc_test=97.17
Run 4/10
loss_test=0.18	 acc_test=97.07
Run 5/10
loss_test=0.23	 acc_test=96.29
Run 6/10
loss_test=0.2	 acc_test=96.78
Run 7/10
loss_test=0.21	 acc_test=96.68
Run 8/10
loss_test=0.15	 acc_test=97.75
Run 9/10
loss_test=0.15	 acc_test=97.17
Run 10/10
loss_test=0.2	 acc_test=96.29


In [6]:
# Print results
results = torch.Tensor(results)
print("Average accuracy:   {:.2f}".format(results.mean()))
print("Standard deviation: {:.2f}".format(results.std()))

Average accuracy:   96.95
Standard deviation: 0.46
